In [57]:
#http://jmcauley.ucsd.edu/data/amazon/
FILE_PATH="./dataset/reviews_Toys_and_Games_5.json"
import json

def parse(path):
    f=open(path,'r')
    for l in f:
        yield json.loads(l)
        
data=[]
count=0
for line in parse(FILE_PATH):
    di=dict()
    di['user']=line['reviewerID']
    di['item']=line['asin']
    di['rating']=line['overall']
    di['review']=line['reviewText']
    data.append(di)
    count+=1
    if count>5000:
        break

In [58]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [109]:
vocabulary={}
count=0
max_length=0
text=""
for d in data:
    if(len(d['review'].split())>max_length):
        max_length=len(d['review'])
        text=d['review']
    for word in d['review'].split():
        if word not in vocabulary.keys():
            vocabulary[word]=count
            count+=1
            

In [130]:
#固定参数
WORD_PER_SENT=20 #每句话的词个数
SENT_PER_REVIEW=10 #每篇review的句子个数
REVIEW_PER_OBJECT=15 #每个user/item的review个数
WORD_EMBEDDING_DIM=300 #每个Word表示的维度
VOCAB_SIZE=len(vocabulary) #总的词语种类
WORD_LEVEL_CNN_FILTER=200 #CNN word level
WORD_LEVEL_CNN_WIN_SIZE=3
SENT_LEVEL_CNN_FILTER=100 #CNN sentence level
SENT_LEVEL_CNN_WIN_SIZE=3

In [131]:
#word embedding
embeds=nn.Embedding(VOCAB_SIZE,WORD_EMBEDDING_DIM)

In [134]:
#每个user/item的reviews
reviews=torch.randint(0,VOCAB_SIZE,(REVIEW_PER_OBJECT,SENT_PER_REVIEW,WORD_PER_SENT))
word_embed=embeds(reviews)
word_embed.size()

torch.Size([15, 10, 20, 300])

In [135]:
word_level_cnn=nn.Conv1d(WORD_EMBEDDING_DIM,#in_channels
                         WORD_LEVEL_CNN_FILTER,#out_channels
                         WORD_LEVEL_CNN_WIN_SIZE,#kernel_size
                         padding=SENT_LEVEL_CNN_WIN_SIZE//2)

In [159]:
word_before_attention=word_level_cnn(word_embed.view(-1,WORD_PER_SENT,WORD_EMBEDDING_DIM).permute(0,2,1)).permute(0,2,1).view(REVIEW_PER_OBJECT,SENT_PER_REVIEW,WORD_PER_SENT,WORD_LEVEL_CNN_FILTER)
word_before_attention.size()

torch.Size([15, 10, 20, 200])

In [214]:
word_level_attention=nn.Linear(WORD_LEVEL_CNN_FILTER,1,True)

In [229]:
#w_attention=word_level_attention(word_before_attention.contiguous().view(-1,WORD_LEVEL_CNN_FILTER)).view(REVIEW_PER_OBJECT,SENT_PER_REVIEW,WORD_PER_SENT,1)
w_attention=torch.tanh(word_level_attention(word_before_attention))
w_attention.size()

torch.Size([15, 10, 20, 1])

In [216]:
w_attention2=F.softmax(w_attention,2)
w_attention2.size()

torch.Size([15, 10, 20, 1])

In [217]:
sentence=torch.sum(w_attention2*word_before_attention,dim=2)
sentence.size()

torch.Size([15, 10, 200])

In [218]:
sent_level_cnn=nn.Conv1d(WORD_LEVEL_CNN_FILTER,SENT_LEVEL_CNN_FILTER,SENT_LEVEL_CNN_WIN_SIZE,padding=SENT_LEVEL_CNN_WIN_SIZE//2)

In [219]:
sent_before_attention=sent_level_cnn(sentence.permute(0,2,1)).permute(0,2,1)
sent_before_attention.size()

torch.Size([15, 10, 100])

In [220]:
sent_level_attention=nn.Linear(SENT_LEVEL_CNN_FILTER,1,True)

In [228]:
s_attention=torch.tanh(sent_level_attention(sent_before_attention))
s_attention.size()

torch.Size([15, 10, 1])

In [222]:
s_attenion2=F.softmax(s_attention,1)
s_attenion2.size()

torch.Size([15, 10, 1])

In [225]:
object_embed_before_attention=torch.sum(s_attenion2*sent_before_attention,dim=1)
object_embed_before_attention.size()

torch.Size([15, 100])

In [224]:
review_level_attention=nn.Linear(SENT_LEVEL_CNN_FILTER,1,True)

In [227]:
r_attention=torch.tanh(review_level_attention(object_embed_before_attention))
r_attention.size()

torch.Size([15, 1])

In [232]:
r_attention2=F.softmax(r_attention,dim=0)
r_attention2.size()

torch.Size([15, 1])

In [237]:
object_embed=torch.sum(r_attention2*object_embed_before_attention,dim=0)
object_embed.size()

torch.Size([100])

In [268]:
class HUITA_embed_nn(nn.Module):
    def __init__(self):
        super(HUITA_embed_nn,self).__init__()
        self.embeds=nn.Embedding(VOCAB_SIZE,WORD_EMBEDDING_DIM)
        self.word_level_cnn=nn.Conv1d(WORD_EMBEDDING_DIM,WORD_LEVEL_CNN_FILTER,WORD_LEVEL_CNN_WIN_SIZE,padding=SENT_LEVEL_CNN_WIN_SIZE//2)
        self.word_level_attention=nn.Linear(WORD_LEVEL_CNN_FILTER,1,True)
        self.sent_level_cnn=nn.Conv1d(WORD_LEVEL_CNN_FILTER,SENT_LEVEL_CNN_FILTER,SENT_LEVEL_CNN_WIN_SIZE,padding=SENT_LEVEL_CNN_WIN_SIZE//2)
        self.sent_level_attention=nn.Linear(SENT_LEVEL_CNN_FILTER,1,True)
        self.review_level_attention=nn.Linear(SENT_LEVEL_CNN_FILTER,1,True)
        
    
    def forward(self,reviews):
        word_embed=self.embeds(reviews)
        word_before_attention=self.word_level_cnn(word_embed.view(-1,WORD_PER_SENT,WORD_EMBEDDING_DIM).permute(0,2,1)).permute(0,2,1).view(REVIEW_PER_OBJECT,SENT_PER_REVIEW,WORD_PER_SENT,WORD_LEVEL_CNN_FILTER)
        w_attention=torch.tanh(self.word_level_attention(word_before_attention))
        w_attention2=F.softmax(w_attention,2)
        sentence=torch.sum(w_attention2*word_before_attention,dim=2)
        sent_before_attention=self.sent_level_cnn(sentence.permute(0,2,1)).permute(0,2,1)
        s_attention=torch.tanh(self.sent_level_attention(sent_before_attention))
        s_attenion2=F.softmax(s_attention,1)
        object_embed_before_attention=torch.sum(s_attenion2*sent_before_attention,dim=1)
        r_attention=torch.tanh(self.review_level_attention(object_embed_before_attention))
        r_attention2=F.softmax(r_attention,dim=0)
        object_embed=torch.sum(r_attention2*object_embed_before_attention,dim=0)
        return object_embed

In [269]:
class HUITA_predict(nn.Module):
    def __init__(self):
        super(HUITA_predict,self).__init__()
        self.prediction_nn=nn.Linear(SENT_LEVEL_CNN_FILTER,1,True)
        
    def forward(self,user_reviews,item_reviews):
        embed=HUITA_embed_nn()
        user_embed=embed(user_reviews)
        item_embed=embed(item_reviews)
        rating=F.relu(self.prediction_nn(user_embed*item_embed))
        return rating

In [270]:
losses = []
loss_function = nn.MSELoss()
model = HUITA_predict()
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(10):
    model.zero_grad()
    predict_r=model(reviews,reviews)
    loss = loss_function(predict_r, torch.tensor([4.9],dtype=torch.float)) #just demo
    losses.append(loss)

In [271]:
losses

[tensor(24.0100, grad_fn=<MseLossBackward>),
 tensor(24.0100, grad_fn=<MseLossBackward>),
 tensor(24.0100, grad_fn=<MseLossBackward>),
 tensor(24.0100, grad_fn=<MseLossBackward>),
 tensor(24.0100, grad_fn=<MseLossBackward>),
 tensor(24.0100, grad_fn=<MseLossBackward>),
 tensor(24.0100, grad_fn=<MseLossBackward>),
 tensor(24.0100, grad_fn=<MseLossBackward>),
 tensor(24.0100, grad_fn=<MseLossBackward>),
 tensor(24.0100, grad_fn=<MseLossBackward>)]

In [272]:
x=torch.tensor([1,2,3])
x*x

tensor([1, 4, 9])

In [70]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
[517.6566867828369, 515.2297914028168, 512.8179616928101, 510.4198820590973, 508.0348553657532, 505.66215205192566, 503.30089378356934, 500.9498529434204, 498.6096065044403, 496.279981136322]


In [33]:
x=torch.randn(2,3)
y=torch.randn(2,5)
z=torch.cat([x,y],1)